In [1]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

%matplotlib inline
import matplotlib.pyplot as plt
from collections import Counter, defaultdict

from sklearn.model_selection import KFold

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import normalize
import random
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

import random

random.seed(1)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Huynh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Huynh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Huynh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
with open('train.dat', 'r', encoding='utf8') as fh:
    lines = fh.readlines()
len(lines)

14438

In [3]:
with open('test.dat', 'r', encoding='utf8') as fh:
    lines_t = fh.readlines()

In [4]:
docs = [l[2:] for l in lines] + [l for l in lines_t]
y = [l[0] for l in lines]
print(len(docs))
print(len(y))
  

regTok = RegexpTokenizer(r'[a-zA-Z]+')
# regTok = RegexpTokenizer(r'\w+')
docss = []
for i in docs:
    i = i.lower()
    docss.append(regTok.tokenize(i))


s_wrds = set(stopwords.words('english'))
lmtzr = WordNetLemmatizer()
stmm = PorterStemmer()
docsss = []
for i in docss:
    temp = [lmtzr.lemmatize(w) for w in i if not w in s_wrds]
#     temp = [stmm.stem(w) for w in i if not w in s_wrds]
    docsss.append(temp)

docs = []
for i in docsss:
    temp = ''
    for j in i:
        temp = temp + j + ' '
    docs.append(temp[0:-1])

docss = []
docsss = []


c1=0
c2=0
c3=0
c4=0
c5=0
for i in y:
    if int(i) == 1: c1+=1
    elif int(i) == 2: c2+=1
    elif int(i) == 3: c3+=1
    elif int(i) == 4: c4+=1
    elif int(i) == 5: c5+=1
print(c1,c2,c3,c4,c5)

28880
14438
3163 1494 1925 3051 4805


In [5]:
def ryan_preprocess(data, ngram=3, max_df=0.6, min_df=1, stop_words = 'english'):
    
    vect = TfidfVectorizer(max_features=50000, stop_words=stop_words, max_df=max_df, min_df=min_df, ngram_range=(1, ngram), norm='l2', token_pattern='\w{4,}')
    vect.fit(data)
    print(len(vect.get_feature_names()))
#     print(vect.get_feature_names())
    
    csr_mat = vect.transform(data[0:14438])
    csr_mat1 = vect.transform(data[14438:])
    
#     csr_mat = vect.transform(data[0:17857])
#     csr_mat1 = vect.transform(data[17857:])
    print(type(csr_mat))
    
    return csr_mat, csr_mat1
    
def classifyNames(mat,cls, k=3, d=10):
    r""" Classify names using c-mer frequency vector representations of the names and kNN classification with 
    cosine similarity and 10-fold cross validation
    """

    
    def classify(x, train, clstr):
        r""" Classify vector x using kNN and majority vote rule given training data and associated classes
        """
        # find nearest neighbors for x

        dots = x.dot(train.T)

        sims = list(zip(dots.indices, dots.data))
        if len(sims) == 0:
            # could not find any neighbors
            return '+' if np.random.rand() > 0.5 else '-'
        sims.sort(key=lambda x: x[1], reverse=True)
        tc = Counter(clstr[s[0]] for s in sims[:k]).most_common(5)
# weighting the low count classes      
#         for i in range(len(tc)):
#             if tc[i][0] == '2': 
#                 tc[i] =  (tc[i][0], int(tc[i][1] * 1.4))
#             elif tc[i][0] == '3': 
#                 tc[i] =  (tc[i][0], int(tc[i][1] * 1.2))
                
        if len(tc) < 2 or tc[0][1] > tc[1][1]:
            # majority vote
            return tc[0][0]
        # tie break
        tc = defaultdict(float)
        for s in sims[:k]:
            tc[clstr[s[0]]] += s[1]
        return sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0]
        
    macc = 0.0
    mf1 = 0.0
    
    kfold = KFold(d)
    for train, test in kfold.split(mat):
        clstr = []
        clste = []
        
        for i in train:
            clstr.append(y[i])    
        for i in test:
            clste.append(y[i])
        
        train = mat[train]
        test = mat[test]

        # predict the class of each test sample
        clspr = [ classify(test[i,:], train, clstr) for i in range(test.shape[0]) ]
        
        acc = accuracy_score(clste, clspr)
        print('acc: ', acc)
        macc += acc
        
        f1 = f1_score(clste, clspr, average='weighted')
        print('f1: ', f1)
        mf1 += f1
        
    return macc/d, mf1/d


def get_pred(mat,mat1,cls, k=3, d=10):
    r""" Classify names using c-mer frequency vector representations of the names and kNN classification with 
    cosine similarity and 10-fold cross validation
    """
    
    def classify(x, train, clstr):
        r""" Classify vector x using kNN and majority vote rule given training data and associated classes
        """
        # find nearest neighbors for x

        dots = x.dot(train.T)

        sims = list(zip(dots.indices, dots.data))
        if len(sims) == 0:
            # could not find any neighbors
            return '+' if np.random.rand() > 0.5 else '-'
        sims.sort(key=lambda x: x[1], reverse=True)
        tc = Counter(clstr[s[0]] for s in sims[:k]).most_common(5)
        if len(tc) < 2 or tc[0][1] > tc[1][1]:
            # majority vote
            return tc[0][0]
        # tie break
        tc = defaultdict(float)
        for s in sims[:k]:
            tc[clstr[s[0]]] += s[1]
        return sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0]
    
    # predict the class of each test sample
    clspr = [ classify(mat1[i,:], mat, cls) for i in range(mat1.shape[0]) ]
    return clspr

In [7]:
csr_mat, csr_mat1 = ryan_preprocess(docs, max_df=0.7, ngram=6, min_df=1, stop_words='english')

k = 40
acc, f1_score = classifyNames(csr_mat,y, k = k, d = 10)
print("average acc: ", acc)
print("average f1: ", f1_score)
print('================')

50000
<class 'scipy.sparse.csr.csr_matrix'>
acc:  0.6094182825484764
f1:  0.6022179453396013
acc:  0.6121883656509696
f1:  0.6049132590842182
acc:  0.6378116343490304
f1:  0.63170407799652
acc:  0.6253462603878116
f1:  0.6172254907576681
acc:  0.6135734072022161
f1:  0.6044820187750563
acc:  0.6142659279778393
f1:  0.606179492750909
acc:  0.6135734072022161
f1:  0.6076130474409603
acc:  0.6087257617728532
f1:  0.6002989682050679
acc:  0.6292446292446292
f1:  0.6232704824211004
acc:  0.6008316008316008
f1:  0.5890381904671824
average acc:  0.6164979277167643
average f1:  0.6086942973238283


In [ ]:
k = 40
csr_mat, csr_mat1 = ryan_preprocess(docs, max_df=0.7, ngram=6, min_df=1, stop_words='english')

pred = get_pred(csr_mat,csr_mat1, y, k = k)


with open("result_good.dat", "w") as f: 
    for i in pred:
        f.write(i)
        f.write('\n')